In [1]:
import pandas as pd
from pandas import DataFrame, Series

import os

import warnings
warnings.filterwarnings('ignore')

import time
import timeit

import joblib
#import ENG
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.coherencemodel import CoherenceModel
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

import pythainlp
from pythainlp import word_tokenize

import re

def splitword(text):
  out=[]
  couUp=0
  left=0
  initial=text.strip()
  upper=re.findall('[A-Z]',initial)
  for i in range(len(initial)):
    if initial[i] in upper:
      couUp+=1
    if couUp==len(upper):
      out.append(initial[left:i])
      out.append(initial[i:])
      break
    elif initial[i] in upper and i !=0:
      out.append(initial[left:i])
      left=i
  return ' '.join(out).strip()

#Data Pre-processing for English only
# input = many words
#output = [w1, w2, w3, ......]

#1. Tokenization: Split the text into sentences and the sentences into words.
# Lowercase the words and remove punctuation.
#2. Words that have fewer than 3 characters are removed.
#3. All stopwords are removed.
#4. Words are lemmatized — 
#words in third person are changed to first person and verbs in past and future tenses are changed into present.
#5. Words are stemmed — words are reduced to their root form.
def lemmatize_stemming(text):
    return SnowballStemmer('english').stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

#process for all data
#input = docs
# output = a list of each word
def preprocessAll(text):
  text=str(text)
  text=text.strip()
  thai=pythainlp.thai_characters
  th=''
  en=''
  for i in range(len(text)):
    if text[i] in thai:
      th=th+text[i]
    else:
      if (i != 0) and (text[i-1] in thai):
        en=en+' '+text[i]
      else:
        en=en+text[i]
  temp_en=[]
  for i in en.split(' '):
    if i !='':
      temp_en.append(splitword(i))
  temp_en=' '.join(temp_en)
  return preprocess(temp_en)+word_tokenize(th, keep_whitespace=False)

def predict_tfid_loadedModel(text, loaded_model):
  process_unseen= preprocessAll(text)
  dic_unseen=loaded_model.id2word
  bow_unseen=dic_unseen.doc2bow(process_unseen) 
  target=bow_unseen
  return sorted(loaded_model[target], key= lambda x:x[1], reverse= True)

def result(text, rec_main, rec_mid, rec_lowmid, rec_overfit):
    def cal(text, rec):
        out='Others'
        maxy=0
        for item in rec:
            t= predict_tfid_loadedModel(text, item[0])
            if len(t) >0:
                if str(t[0][0]) in item[2]:
                    if t[0][1]>maxy:
                        out= item[1]
                        maxy=t[0][1]
        return out
    
    if cal(text, rec_main) != 'Others':
        return cal(text, rec_main)
    elif cal(text, rec_mid) != 'Others':
        return cal(text, rec_mid)
    elif cal(text, rec_lowmid) != 'Others':
        return cal(text, rec_lowmid)
    else:
        return cal(text, rec_overfit)
start = timeit.default_timer()
''' # load selected model '''
select=pd.read_csv('/home/std/Downloads/csv_file/model_selection_orderTii_V2.csv')
rec_main=[]
rec_mid=[]
rec_lowmid=[]
rec_overfit=[]

### modify model
mid=[1, 2, 3, 5,10, 12,16,25]
lowmid=[14, 23, 24]
overfit=[13]
eliminate=[18, 26, 27, 28, 29]

model_path='/home/std/Downloads/python/model_temp/Final_title_noBiTri_varyTopic_dictionaryV3_AlphaEtaV1/'
#all_model=os.listdir(model_path)
for num in sorted(list(select['order'].unique())):
    #print(num)
    job=select[select['order']==num]['job'].iloc[0]
    #print(job)
    alp=select[select['order']==num]['alpha'].iloc[0]
    #print(alp)
    beta=select[select['order']==num]['beta'].iloc[0]
    #print(beta)
    n_topic=select[select['order']==num]['n_topic'].iloc[0]
    #print(n_topic)
    rep=re.findall('[0-9]{1,3}', select[select['order']==num]['Represent_topic'].iloc[0])
    #print(rep)
    model=joblib.load(model_path+'[below:topic:alpha:eta]_15:'+str(n_topic)+':'+str(alp)+':'+str(beta)+'.sav')
    if num in overfit:
        rec_overfit.append([model, job, rep])
    elif num in lowmid:
        rec_lowmid.append([model, job, rep])
    elif num in mid:
        rec_mid.append([model, job, rep])
    else:
        if num not in eliminate:
            rec_main.append([model, job, rep])
    

    
test=pd.read_csv('/home/std/Downloads/csv_file/test_700kminus6261_combine.csv')[:90000]
print('len test', len(test))
keep=[]
for i in range(len(test)):
    keep.append(result(test.iloc[i]['combine'], rec_main, rec_mid, rec_lowmid, rec_overfit))
test['model_tag']=pd.Series(keep, index=test.index)
test.to_csv('/home/std/Downloads/python/test_result/piplineV2_overfit/pipelineV2_part1.csv')


stop = timeit.default_timer()
print('runtime: ', stop - start)

[nltk_data] Downloading package wordnet to /home/std/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


len test 90000
runtime:  4069.1891041529598


In [10]:
test.head()

,index,_id,title_en,title_th,description,company,combine,model_tag
0,0,5cfe0339ae04a37efca33a0f,NaN,Shop Manager - Luxury Brand,OverviewWe are looking for an experiencedShop ...,"Qapstone Consulting Group Recruitment Co., Ltd.",Shop Manager - Luxury Brand OverviewWe are lo...,retail manager
1,1,5cfe0339ae04a37efca33a0d,NaN,Accounting Manager,Accounting ManagerJob descriptionManage balanc...,"THE MAKE-IT CO., LTD.",Accounting Manager Accounting ManagerJob desc...,Investor // finance // account
2,2,5cfe0339ae04a37efca33a0e,NaN,HR Officer / เจ้าหน้าที่ทรัพยากรมนุษย์,รายละเอียดงาน:ควบคุมอัตรากำลัง งานสรรหาและว่าจ...,"Sri Trang Rubber & Plantation Co., Ltd.",HR Officer / เจ้าหน้าที่ทรัพยากรมนุษย์ รายละเ...,HR
3,3,5cfe0339ae04a37efca33a10,NaN,พนักงานสินค้า/ส่งเอกสารประจำ อำเภอเมือง จังหวั...,Responsibilitiesส่งเอกสารที่ได้รับหมอบหมายให้ส...,"U Delivery Co., Ltd.",พนักงานสินค้า/ส่งเอกสารประจำ อำเภอเมือง จังหว...,coordinator
4,4,5cfe0339ae04a37efca33a0c,NaN,Overseas Buyer/จัดซื้อ(ต่างประเทศ),Job main duties:Prepare costing and control ma...,"Sybarite Co., Ltd.",Overseas Buyer/จัดซื้อ(ต่างประเทศ) Job main d...,Stock // procurement // supply


In [11]:
test['model_tag'].unique()

array(['retail manager', 'Investor // finance // account', 'HR',
       'coordinator', 'Stock // procurement // supply', 'sale // service',
       'marketing // advertising', 'cooker + bartender', 'graphic design',
       'Policy manager // BD', 'waiter', 'translator', 'Others', 'writer',
       'vehicle driver', 'manufacturing manager', 'credit analyst',
       'maid', 'law', 'security + safety', 'teacher', 'therapist',
       'cashier', 'architecture', 'care taker'], dtype=object)

In [12]:
#test.to_csv('/home/std/Downloads/python/test_result/piplineV2_overfit/pipelineV2_part1.csv')

In [21]:
type(test.iloc[0]['model_tag'])

list

In [24]:
for j in test.iloc[0]['model_tag']:
    print(type(j))

<class 'list'>
